In [ ]:
####################################################
import matplotlib.pyplot as plt
import matplotlib
####################################################
import scipy.fftpack
from scipy.signal import find_peaks
####################################################
import numpy as np
import csv
####################################################
import pandas as pd
from pandas import DataFrame
import pandas_profiling
from pandas_profiling import ProfileReport
####################################################
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import plot
import plotly.io as pio
#import kaleido
####################################################
from datetime import datetime
import datetime as dt
from datetime import timedelta
####################################################
import math
import operator
from operator import methodcaller
####################################################
from docx import Document
from docx.shared import Inches
from docx2pdf import convert
####################################################
import os
import pathlib

#%matplotlib widget
#%matplotlib inline

# Variable Setup

In [ ]:
StepRes = 200   # Resolution of steps for all series
B = 100         # Engine Vertical Gimbal - TVC mount length (mm)
# engine size loop limit is 50 due to inefficient code
# it does work for StepRes>50 it just takes substantually longer
ThetaA = 90     # Degree angle of A-C at netual engine position
EngMaxTheta = 10 # +- Engine actuation angle

Rlow = 1.01     # Ratio of A-B Low point
Rhigh = 2.00    # Ratio of A-B High point
Bstep = 1       # Step increase of Demension B (mm)


print(str(B+StepRes*Bstep))


# script file

In [ ]:
filename = ("Eng_Pitch_Analyis_for_Eng-Bay-ACT_B="+str(B)+"-"+str(B+StepRes*Bstep)+"(mm)")                           # copy file name

Open_Directory = r"-- USER MUST ENTER PRIMARY FOLDER LOCATION --"                            # import file folder --- primary location
Subfolder = r"Simulation\\Engine_Dynamics\\"
FileAddress  = Open_Directory + Subfolder    # create full file address

try: os.makedirs(FileAddress); print ("Directory is primary location")                                                # check Directory address is correct
except: Open_Directory = r"-- USER MUST ENTER SECONDARY FOLDER LOCATION --"; print ("Directory is PC")       # import file folder --- alternative location 
    
FileAddress  = Open_Directory + Subfolder    # create full file address

print(FileAddress)

## file data

In [ ]:
now = datetime.now()
DataTime = now.strftime ("%Y-%m-%d, %H-%M")

DisplayGraphsInBrowser = True
DisplayGraphsInSeries = False
SaveData = True

if SaveData == True:
    newpath = FileAddress + str(DataTime) +"_" + filename
    if not os.path.exists(newpath):
        os.makedirs(newpath)

    print(newpath)

## import raw data from file

# Loop DataFrame

In [ ]:
df=[]
# Creation of Loop DataFrame with an intial sort
LoopDF = DataFrame(df)#, columns=[""].astype(float))
LoopDF = LoopDF[sorted(LoopDF)]
##########################################################
# Creation of Global DataFrame with an intial sort
GlobalDF = DataFrame(df)#, columns=[""].astype(float))
GlobalDF = GlobalDF[sorted(GlobalDF)]
##########################################################

LoopDF
GlobalDF

GlobalDF["Lowest Angular Pitch (Deg/mm)"]=[]
GlobalDF["A-Neutral @ Lowest Angular Pitch (mm)"]=[]
GlobalDF["C @ Lowest Angular Pitch (mm)"]=[]

Pitch00=[]
A00=[]
C00=[]

####

GlobalDF["0.5 Angular Pitch (Deg/mm)"]=[]
GlobalDF["A-Neutral @ 0.5 Angular Pitch (mm)"]=[]
GlobalDF["C @ 0.5 Angular Pitch (mm)"]=[]

Pitch05=[]
A05=[]
C05=[]

####

GlobalDF["1.0 Angular Pitch (Deg/mm)"]=[]
GlobalDF["A-Neutral @ 1.0 Angular Pitch (mm)"]=[]
GlobalDF["C @ 1.0 Angular Pitch (mm)"]=[]

Pitch10=[]
A10=[]
C10=[]

# Loop Calculation

In [ ]:
k = 0
Bstart =B

for k in range(StepRes):
    LoopDF["A_(mm)-Neutral"] = np.linspace(Rlow, Rhigh, StepRes)*B      # Series of possible actuator legnths as a ratio of B (mm)

    LoopDF["C_(mm)"] = (LoopDF["A_(mm)-Neutral"]**2 - B**2)**0.5  # Engine Horizontal Gimbal - TVC mount length (mm)
    LoopDF["thetaB-Neutral"] = LoopDF["A_(mm)-Neutral"]
    LoopDF["thetaC-Neutral"] = LoopDF["A_(mm)-Neutral"]
  
    i=0
    
    for i in range(len(LoopDF["A_(mm)-Neutral"])):
        try: LoopDF["thetaB-Neutral"][i] = math.degrees(math.asin(B/LoopDF["A_(mm)-Neutral"][i]))
        except: print("dataseries not inluced")

        try: LoopDF["thetaC-Neutral"][i] = math.degrees(math.asin(LoopDF["C_(mm)"][i]/LoopDF["A_(mm)-Neutral"][i]))
        except: print("dataseries not inluced")
        i += 1

    LoopDF=LoopDF.round(2) 

    LoopDF["EngMaxThetaRange_0"] = np.linspace(-EngMaxTheta, EngMaxTheta, StepRes)
    LoopDF["EngMaxThetaRange_90"] = 90 + LoopDF["EngMaxThetaRange_0"]  

    i=0
    for i in range(len(LoopDF["A_(mm)-Neutral"])):
        LoopDF["TVCActLength" +"_A="+ str(LoopDF["A_(mm)-Neutral"][i])+"(mm)"] = LoopDF["A_(mm)-Neutral"]

    j=0
    i=0

    for j in range(len(LoopDF["A_(mm)-Neutral"])):  
        for i in range(len(LoopDF["A_(mm)-Neutral"])):
            try: LoopDF["TVCActLength" +"_A="+ str(LoopDF["A_(mm)-Neutral"][j])+"(mm)"][i] = ((B**2 + LoopDF["C_(mm)"][j]**2 -(2*B*LoopDF["C_(mm)"][j]) * math.cos(math.radians(LoopDF["EngMaxThetaRange_90"][i])))**0.5- LoopDF["A_(mm)-Neutral"][j])
            except: print("dataseries not inluced")

            i += 1
            
        j += 1

    # # Calculation to score the performance of a TVC layout design based on the
    # # gradient constant of the relationship between TVC actuation length and engine angle

    i=0
    for i in range(len(LoopDF["A_(mm)-Neutral"])):
        LoopDF["Angular Pitch (Deg/mm)"] = LoopDF["A_(mm)-Neutral"]


    i=0

    for i in range(len(LoopDF["A_(mm)-Neutral"])):
        try: LoopDF["Angular Pitch (Deg/mm)"][i] = ((max(LoopDF["EngMaxThetaRange_90"]) - min(LoopDF["EngMaxThetaRange_90"]))/(max(LoopDF["TVCActLength" +"_A="+ str(LoopDF["A_(mm)-Neutral"][i])+"(mm)"]-min(LoopDF["TVCActLength" +"_A="+ str(LoopDF["A_(mm)-Neutral"][i])+"(mm)"]))))
        except: print("dataseries not inluced")

        i += 1

    ####
    # DataFrame is highly fragmented.
    # This is usually the result of calling `frame.insert` many times, which has poor performance. 
    # Consider joining all columns at once using pd.concat(axis=1) instead.
    # To get a de-fragmented frame, use `newframe = frame.copy()`
    ####
    Apitch_00 = LoopDF["Angular Pitch (Deg/mm)"].sub(0).abs().idxmin()
    Pitch00.append(LoopDF["Angular Pitch (Deg/mm)"][Apitch_00].round(5))
    A00.append(LoopDF["A_(mm)-Neutral"][Apitch_00].round(5))
    C00.append(LoopDF["C_(mm)"][Apitch_00].round(5))

    ####
    Apitch_05 = LoopDF["Angular Pitch (Deg/mm)"].sub(0.5).abs().idxmin()
    Pitch05.append(LoopDF["Angular Pitch (Deg/mm)"][Apitch_05].round(5))
    A05.append(LoopDF["A_(mm)-Neutral"][Apitch_05].round(5))
    C05.append(LoopDF["C_(mm)"][Apitch_05].round(5))

    ####
    Apitch_10 = LoopDF["Angular Pitch (Deg/mm)"].sub(1.0).abs().idxmin()
    Pitch10.append(LoopDF["Angular Pitch (Deg/mm)"][Apitch_10].round(5))
    A10.append(LoopDF["A_(mm)-Neutral"][Apitch_10].round(5))
    C10.append(LoopDF["C_(mm)"][Apitch_10].round(5))


    ####
    B = B+Bstep   

GlobalDF["Lowest Angular Pitch (Deg/mm)"] = Pitch00
GlobalDF["A-Neutral @ Lowest Angular Pitch (mm)"] = A00
GlobalDF["C @ Lowest Angular Pitch (mm)"] = C00
#
GlobalDF["0.5 Angular Pitch (Deg/mm)"] = Pitch05
GlobalDF["A-Neutral @ 0.5 Angular Pitch (mm)"] = A05
GlobalDF["C @ 0.5 Angular Pitch (mm)"] = C05
#
GlobalDF["1.0 Angular Pitch (Deg/mm)"] = Pitch10
GlobalDF["A-Neutral @ 1.0 Angular Pitch (mm)"] = A10
GlobalDF["C @ 1.0 Angular Pitch (mm)"] = C10
#
GlobalDF["B (mm)"] = np.linspace(Bstart, (B), StepRes)
#
GlobalDF

# Plot_2D Interactive 
##### Pitch

In [ ]:
# GlobalDF["0.5 Angular Pitch (Deg/mm)"] = Pitch05
# GlobalDF["A-Neutral @ 0.5 Angular Pitch (mm)"] = A05
# GlobalDF["C @ 0.5 Angular Pitch (mm)"] = C05
# #
# GlobalDF["B (mm)"] = np.linspace(Bstart, (B), StepRes)
# #


# Setup Plot
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.update_layout(legend_orientation="v", title="<b>"+str(filename)+"_Against Pitch"+"<b>", title_font_size = 24, legend_font_size=22)
# + " - Angular Pitch for B= "+str(Bstart)+"-"+str(B)+" (mm)" 
#fig.update_layout(height=1920, width = 1080, legend_orientation="v", title="<b>"+str(file)+"<b>", title_font_size = 24)
fig.update_yaxes(title_text = "TVC Layout Dimension Length (mm)", title_font_size=24, row=1, col =1)
fig.update_xaxes(title_text = "Angular Pitch (Deg/mm)", title_font_size=24, row=1, col =1)
fig.update_layout(legend_borderwidth = 4)


# Plot GlobalDF
## group for clarity
##########################################################
try: fig.add_trace(go.Scatter(x=GlobalDF["Lowest Angular Pitch (Deg/mm)"], y=GlobalDF["A-Neutral @ Lowest Angular Pitch (mm)"], mode="lines", name ="A-Neutral @ Lowest Angular Pitch (mm)"))
except: print("dataseries not incluced")
try: fig.add_trace(go.Scatter(x=GlobalDF["Lowest Angular Pitch (Deg/mm)"], y=GlobalDF["C @ Lowest Angular Pitch (mm)"], mode="lines", name ="C @ Lowest Angular Pitch (mm)"))
except: print("dataseries not incluced")
##########################################################
try: fig.add_trace(go.Scatter(x=GlobalDF["0.5 Angular Pitch (Deg/mm)"], y=GlobalDF["A-Neutral @ 0.5 Angular Pitch (mm)"], mode="lines", name ="A-Neutral @ 0.5 Angular Pitch (mm)"))
except: print("dataseries not incluced")
try: fig.add_trace(go.Scatter(x=GlobalDF["0.5 Angular Pitch (Deg/mm)"], y=GlobalDF["C @ 0.5 Angular Pitch (mm)"], mode="lines", name ="C @ 0.5 Angular Pitch (mm)"))
except: print("dataseries not incluced")
##########################################################
try: fig.add_trace(go.Scatter(x=GlobalDF["1.0 Angular Pitch (Deg/mm)"], y=GlobalDF["A-Neutral @ 1.0 Angular Pitch (mm)"], mode="lines", name ="A-Neutral @ 1.0 Angular Pitch (mm)"))
except: print("dataseries not incluced")
try: fig.add_trace(go.Scatter(x=GlobalDF["1.0 Angular Pitch (Deg/mm)"], y=GlobalDF["C @ 1.0 Angular Pitch (mm)"], mode="lines", name ="C @ 1.0 Angular Pitch (mm)"))
except: print("dataseries not incluced")
##########################################################
#import imgkit

# Display Graph
if DisplayGraphsInBrowser == True:
    # plot(fig,show_link = True, filename = newpath+"\\"+"Angular_Pitch_for_B="+str(Bstart)+"-"+str(B)+"(mm)"+".html")
    # pio.write_image(fig, newpath+"\\"+"Angular_Pitch_for_B="+str(Bstart)+"-"+str(B)+"(mm).png", width=1920, height=1080, format="png")
    plot(fig,show_link = True, filename = newpath+"\\"+str(filename)+"_Against Pitch"+".html")
    pio.write_image(fig, newpath+"\\"+str(filename)+"_Against Pitch"+".png", width=1920, height=1080, format="png")
if DisplayGraphsInSeries == True:
    print("display check")
    fig.show
    

#### Plot 2D Dimensions against B

In [ ]:
# Setup Plot
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.update_layout(legend_orientation="v", title="<b>"+str(filename)+"_Against B"+"<b>", title_font_size = 24, legend_font_size=22)
# + " - Angular Pitch for B= "+str(Bstart)+"-"+str(B)+" (mm)" 
#fig.update_layout(height=1920, width = 1080, legend_orientation="v", title="<b>"+str(file)+"<b>", title_font_size = 24)
fig.update_yaxes(title_text = "TVC Layout Dimension Length (mm)", title_font_size=24, row=1, col =1)
fig.update_yaxes(title_text = "Angular Pitch (Deg/mm)", title_font_size=24, row=1, col =1, secondary_y=True)
fig.update_xaxes(title_text = "Engine Vertical Gimbal - TVC mount length (mm)", title_font_size=24, row=1, col =1)
fig.update_layout(legend_borderwidth = 4)


# Plot GlobalDF
## group for clarity
##########################################################
try: fig.add_trace(go.Scatter(x=GlobalDF["B (mm)"], y=GlobalDF["A-Neutral @ Lowest Angular Pitch (mm)"], mode="lines", name ="A-Neutral @ Lowest Angular Pitch (mm)"))
except: print("dataseries not incluced")
try: fig.add_trace(go.Scatter(x=GlobalDF["B (mm)"], y=GlobalDF["C @ Lowest Angular Pitch (mm)"], mode="lines", name ="C @ Lowest Angular Pitch (mm)"))
except: print("dataseries not incluced")
try: fig.add_trace(go.Scatter(x=GlobalDF["B (mm)"], y=GlobalDF["Lowest Angular Pitch (Deg/mm)"], mode="lines", name ="Lowest Angular Pitch (Deg/mm): [Y2-axis]"), secondary_y=True)
except: print("dataseries not incluced")
##########################################################
try: fig.add_trace(go.Scatter(x=GlobalDF["B (mm)"], y=GlobalDF["A-Neutral @ 0.5 Angular Pitch (mm)"], mode="lines", name ="A-Neutral @ 0.5 Angular Pitch (mm)"))
except: print("dataseries not incluced")
try: fig.add_trace(go.Scatter(x=GlobalDF["B (mm)"], y=GlobalDF["C @ 0.5 Angular Pitch (mm)"], mode="lines", name ="C @ 0.5 Angular Pitch (mm)"))
except: print("dataseries not incluced")
try: fig.add_trace(go.Scatter(x=GlobalDF["B (mm)"], y=GlobalDF["0.5 Angular Pitch (Deg/mm)"], mode="lines", name ="0.5 Angular Pitch (Deg/mm): [Y2-axis]"), secondary_y=True)
except: print("dataseries not incluced")
##########################################################
try: fig.add_trace(go.Scatter(x=GlobalDF["B (mm)"], y=GlobalDF["A-Neutral @ 1.0 Angular Pitch (mm)"], mode="lines", name ="A-Neutral @ 1.0 Angular Pitch (mm)"))
except: print("dataseries not incluced")
try: fig.add_trace(go.Scatter(x=GlobalDF["B (mm)"], y=GlobalDF["C @ 1.0 Angular Pitch (mm)"], mode="lines", name ="C @ 1.0 Angular Pitch (mm)"))
except: print("dataseries not incluced")
try: fig.add_trace(go.Scatter(x=GlobalDF["B (mm)"], y=GlobalDF["1.0 Angular Pitch (Deg/mm)"], mode="lines", name ="1.0 Angular Pitch (Deg/mm): [Y2-axis]"), secondary_y=True)
except: print("dataseries not incluced")
##########################################################
#import imgkit

# Display Graph
if DisplayGraphsInBrowser == True:
    # plot(fig,show_link = True, filename = newpath+"\\"+"Angular_Pitch_for_B="+str(Bstart)+"-"+str(B)+"(mm)"+".html")
    # pio.write_image(fig, newpath+"\\"+"Angular_Pitch_for_B="+str(Bstart)+"-"+str(B)+"(mm).png", width=1920, height=1080, format="png")
    plot(fig,show_link = True, filename = newpath+"\\"+str(filename)+"_Against B"+".html")
    pio.write_image(fig, newpath+"\\"+str(filename)+"_Against B"+".png", width=1920, height=1080, format="png")
if DisplayGraphsInSeries == True:
    print("display check")
    fig.show
    

# LoopDF Mean, Min, Max, STD

In [ ]:
# use this section to final the mean, min, max and standard diiviation of all dataseries
# include a final sorting

#=================== Prints Data Summary Table =============================#
DataSummary=GlobalDF
DataSummary = GlobalDF[sorted(DataSummary)]
DataSummary=DataSummary.describe()
#DataSummary=DataSummary.describe(percentiles=[.5])

DataSummary["StepResolution"] = StepRes                 # Resolution of steps for all series
DataSummary["Bstep (mm)"] = Bstep                       # Increment of B in (mm)
DataSummary["ThetaA (Deg)"] = ThetaA                    # Degree angle of A-C at netual engine position
DataSummary["EngMaxTheta (+-Deg)"] = EngMaxTheta        # +- Engine actuation angle
DataSummary["A-B_Ratio_Low"] = Rlow                     # Ratio of A-B Low point
DataSummary["A-B_Ratio_High"] = Rhigh                   # Ratio of A-B High point

try: DataSummary=DataSummary.drop(columns=("x"))
except: print("dataseries not inluced")
DataSummary=DataSummary.round(2)
DataSummary=DataSummary.T

if (SaveData==True):
    DataSummary.to_csv(newpath+"\\"+"AngularPitch_Data_Summary"+".csv")
    GlobalDF.to_csv(newpath+"\\"+"AngularPitch_Analysis_Data"+".csv")
    LoopDF.to_csv(newpath+"\\"+"Loop_Analysis_Data"+".csv")
DataSummary

# Export Report

## Create Report Document in Word

In [ ]:
document = Document(Open_Directory + "\\Test_Report_Templates\\Test_Report_Template_v1.0.docx")   # user must set the correct location of the report template

# the style types must be stored in the doc, word stores them in the program until they are used.
# create a new paragraph and assign it the style, before deleting the paragraph and saving.

document.add_heading("Test_Report" + DataTime,0)

p = document.add_paragraph("This is an auto-generated test report produced by: ")
p.add_run("USER NAME").bold = True



####
document.add_picture(newpath+"\\"+str(filename)+"_Against Pitch"+".png", width=Inches(7))
document.add_picture(newpath+"\\"+str(filename)+"_Against B"+".png", width=Inches(7))
####
document.add_page_break()
####
document.add_heading("Test Data Summary Table", level=1)
document.add_paragraph("The following data is a summary of both the source recorded data series and the processed data", style="Intense Quote")

# add a table to the end and create a reference variable
# extra row is so we can add the header row
t = document.add_table(DataSummary.shape[0]+1, DataSummary.shape[1]+1)
t.style = "Table Grid"

# add the header rows.
for j in range(DataSummary.shape[-1]):
    t.cell(0,j+1).text = DataSummary.columns[j]
    
# include index
t.cell(0, 0).text = "Data Series Index"
for i in range(DataSummary.shape[0]):
    t.cell(i+1, 0).text = DataSummary.index[i]

# add the rest of the data frame
for i in range(DataSummary.shape[0]):
    for j in range(DataSummary.shape[-1]):
        t.cell(i+1,j+1).text = str(DataSummary.values[i,j])
        
# make headers and index bold
def make_rows_bold(*rows):
    for row in rows:
        for cell in row.cells:
            for paragraph in cell.paragraphs:
                for run in paragraph.runs:
                    run.font.bold = True
def make_cols_bold(*columns):
    for col in columns:
        for cell in col.cells:
            for paragraph in cell.paragraphs:
                for run in paragraph.runs:
                    run.font.bold = True

make_rows_bold(t.rows[0])
make_cols_bold(t.columns[0])

document.add_page_break()

document.save(newpath +"\\"+"Test_Report" + DataTime +".docx")

## Convert Word Report to PDF

In [ ]:
# convert("input.docx")
# convert("input.docx", "output.pdf")
# convert("my_docx_folder/")


convert(newpath +"\\"+"Test_Report" + DataTime +".docx", newpath +"\\"+"Test_Report" + DataTime +".pdf")
